In [9]:
import numpy as np
import cv2
from collections import Counter
import os

img_path = 'sar_1_gray.jpg'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

img = img.astype(np.float32)

np.savetxt('mono_image.txt', img, fmt='%d')


In [10]:
def haar_transform(image):
    """Двумерное вейвлет-преобразование Хаара"""
    rows, cols = image.shape
    
    
    row_transform = np.zeros_like(image, dtype=np.float32)
    for i in range(rows):
        for j in range(0, cols - 1, 2):
            avg = (image[i, j] + image[i, j + 1]) / 2
            diff = (image[i, j] - image[i, j + 1]) / 2
            row_transform[i, j // 2] = avg
            row_transform[i, (j // 2) + cols // 2] = diff
    
    result = np.zeros_like(row_transform, dtype=np.float32)
    for j in range(cols):
        for i in range(0, rows - 1, 2):
            avg = (row_transform[i, j] + row_transform[i + 1, j]) / 2
            diff = (row_transform[i, j] - row_transform[i + 1, j]) / 2
            result[i // 2, j] = avg
            result[(i // 2) + rows // 2, j] = diff
    
    LL = result[:rows // 2, :cols // 2]      
    LH = result[:rows // 2, cols // 2:]      
    HL = result[rows // 2:, :cols // 2]     
    HH = result[rows // 2:, cols // 2:]      
    
    return LL, LH, HL, HH

LL, LH, HL, HH = haar_transform(img)
print("Вейвлет-преобразование Хаара выполнено")

Вейвлет-преобразование Хаара выполнено


In [11]:
def quantize(component, n_quants=4):
    """Квантование компоненты в n уровней"""
    min_val = np.min(component)
    max_val = np.max(component)
    
    if max_val == min_val:
        return np.zeros_like(component, dtype=int), min_val, 0
    
    step = (max_val - min_val) / n_quants
    quantized = np.floor((component - min_val) / step).astype(int)
    
    quantized = np.clip(quantized, 0, n_quants - 1)
    
    return quantized, min_val, step

n_quants = 4
LH_q, LH_min, LH_step = quantize(LH, n_quants)
HL_q, HL_min, HL_step = quantize(HL, n_quants)
HH_q, HH_min, HH_step = quantize(HH, n_quants)
print(f"Квантование выполнено с {n_quants} уровнями")

Квантование выполнено с 4 уровнями


In [12]:
def run_length_encode(component):
    """Кодирование длин серий (RLE) для компоненты"""
    return list(Counter(component.flatten()).items())

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)
print("RLE кодирование выполнено")

with open('haar_compressed.txt', 'w') as f:
    np.savetxt(f, LL, fmt='%.2f')
    f.write('\n')
    
    f.write(f"# Quantization params\n")
    f.write(f"LH: {LH_min:.6f} {LH_step:.6f}\n")
    f.write(f"HL: {HL_min:.6f} {HL_step:.6f}\n")
    f.write(f"HH: {HH_min:.6f} {HH_step:.6f}\n")
    f.write('\n')
    
    f.write("# LH component\n")
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')
    
    f.write("# HL component\n")
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")
    f.write('\n')
    
    f.write("# HH component\n")
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")

print("Данные сохранены в файл 'haar_compressed.txt'")

RLE кодирование выполнено
Данные сохранены в файл 'haar_compressed.txt'


In [13]:
def compare_sizes():
    """Сравнение размеров исходного и сжатого изображений"""
    original_size = img.nbytes
    
    if os.path.exists('mono_image.txt'):
        mono_file_size = os.path.getsize('mono_image.txt')
    else:
        mono_file_size = original_size
    
    compressed_size = os.path.getsize('haar_compressed.txt')
    
    print("\n" + "="*50)
    print("СРАВНЕНИЕ ОБЪЕМОВ ПАМЯТИ")
    print("="*50)
    print(f"Исходное изображение (массив в памяти): {original_size:,} байт")
    print(f"Исходное изображение (текстовый файл):  {mono_file_size:,} байт")
    print(f"После Хаара + RLE (сжатый файл):        {compressed_size:,} байт")
    print("-"*50)
    print(f"Коэффициент сжатия (относительно текстового файла): {mono_file_size / compressed_size:.2f}")
    print(f"Экономия памяти: {(1 - compressed_size/mono_file_size)*100:.1f}%")

compare_sizes()
    


СРАВНЕНИЕ ОБЪЕМОВ ПАМЯТИ
Исходное изображение (массив в памяти): 960,000 байт
Исходное изображение (текстовый файл):  765,222 байт
После Хаара + RLE (сжатый файл):        371,238 байт
--------------------------------------------------
Коэффициент сжатия (относительно текстового файла): 2.06
Экономия памяти: 51.5%
